<a href="https://colab.research.google.com/github/yingchenjin/IANAL/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional, for better lemmatization
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure required NLTK data is downloaded
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

# Load the dataset
file_path = "/content/data-ianal.csv"
df = pd.read_csv(file_path)

# Combine question and answer text
df["text"] = df["question_body"].fillna("") + " " + df["answer_body"].fillna("")

# Function for text cleaning
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize words
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]

    # Lemmatize words (reduce to base form)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a single string
    return " ".join(words)

# Apply cleaning function to the text column
df["cleaned_text"] = df["text"].apply(clean_text)

# Save the processed dataset
processed_file_path = "/mnt/data/processed_data_ianal.csv"
df.to_csv(processed_file_path, index=False)

print("Text preprocessing complete! Processed file saved as:", processed_file_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 108.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import spacy
from bs4 import BeautifulSoup

# Load spaCy English NLP model
nlp = spacy.load("en_core_web_sm")

# Load the dataset
file_path = "/content/merged_output.csv"
df = pd.read_csv(file_path)

# Combine question and answer text
df["text"] = df["question_body"].fillna("") + " " + df["answer_body"].fillna("")

# Function for text cleaning
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    # Process with spaCy (tokenization, stopword removal, lemmatization)
    doc = nlp(text)
    words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    # Join words back into a single string
    return " ".join(words)

# Apply cleaning function to the text column
df["cleaned_text"] = df["text"].apply(clean_text)

# Save the processed dataset
processed_file_path = "processed_data_ianal.csv"
df.to_csv(processed_file_path, index=False)

print("Text preprocessing complete! Processed file saved as:", processed_file_path)


Text preprocessing complete! Processed file saved as: processed_data_ianal.csv


In [ ]:
import spacy

try:
    nlp = spacy.load("en_core_web_sm")
    print("spaCy model loaded successfully!")
except OSError:
    print("spaCy model not found. Try reinstalling.")


spaCy model loaded successfully!


In [ ]:
!pip install ace_tools

In [ ]:
import pandas as pd
import spacy
import re
import html
from lxml import etree

# Load spaCy English NLP model
nlp = spacy.load("en_core_web_sm")

# Function to remove HTML tags using lxml
def remove_html(text):
    if pd.isna(text):
        return ""

    # Decode HTML entities (e.g., &lt; becomes <, &amp; becomes &)
    text = html.unescape(text)

    # Parse with lxml to remove tags
    parser = etree.HTMLParser()
    tree = etree.fromstring(f"<root>{text}</root>", parser)
    text = "".join(tree.itertext())

    return text

# Function for advanced text cleaning
def clean_text(text):
    text = remove_html(text)  # First, remove HTML

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    # Convert to lowercase
    text = text.lower()

    # Process text using spaCy (Tokenization, Stopword Removal, Lemmatization)
    doc = nlp(text)
    words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    # Join words back into a single string
    return " ".join(words)

# Load dataset
file_path = "merged_output.csv"
df = pd.read_csv(file_path)

# Combine question and answer text and apply cleaning
df["cleaned_text"] = (df["question_body"].fillna("") + " " + df["answer_body"].fillna("")).apply(clean_text)

# Save the processed dataset
processed_file_path = "processed_merged_ianal.csv"
df.to_csv(processed_file_path, index=False)

  # Save the cleaned dataset locally
print("Processed data saved as processed_merged_ianal.csv")

Processed data saved as processed_merged_ianal.csv


Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load cleaned dataset
file_path = "processed_merged_ianal.csv"  # Use the correct path if needed
df = pd.read_csv(file_path)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 words

# Convert text to TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(df["cleaned_text"])

# Convert to DataFrame for visualization
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Save or print sample results
tfidf_df.to_csv("tfidf_features.csv", index=False)
print("TF-IDF feature extraction complete. Data saved as tfidf_features.csv")


TF-IDF feature extraction complete. Data saved as tfidf_features.csv


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
bow_vectorizer = CountVectorizer(max_features=5000)  # Limit vocabulary size

# Convert text to BoW features
bow_matrix = bow_vectorizer.fit_transform(df["cleaned_text"])

# Convert to DataFrame for visualization
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=bow_vectorizer.get_feature_names_out())

# Save or print sample results
bow_df.to_csv("bow_features.csv", index=False)
print("BoW feature extraction complete. Data saved as bow_features.csv")


BoW feature extraction complete. Data saved as bow_features.csv


In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Tokenize the text (Word2Vec expects tokenized input)
tokenized_texts = df["cleaned_text"].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=5, workers=4)

# Convert each document into a vector (average of word vectors)
def document_vector(doc):
    vectors = [word2vec_model.wv[word] for word in doc if word in word2vec_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

df["word2vec_vector"] = df["cleaned_text"].apply(lambda x: document_vector(x.split()))

# Save or print sample results
df[["word2vec_vector"]].to_csv("word2vec_embeddings.csv", index=False)
print("Word2Vec embeddings extracted. Data saved as word2vec_embeddings.csv")


Word2Vec embeddings extracted. Data saved as word2vec_embeddings.csv


In [ ]:
!pip install gensim scikit-learn


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

# Load cleaned dataset
file_path = "processed_merged_ianal.csv"
df = pd.read_csv(file_path)

# Tokenize text for LDA
tokenized_texts = df["cleaned_text"].apply(lambda x: x.split())

# Create dictionary and corpus for LDA
dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Define number of topics
num_topics = 5  # You can tune this number

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10)

# Display topics with top words
topics = lda_model.print_topics(num_words=10)
for topic_id, topic_words in topics:
    print(f"Topic {topic_id+1}: {topic_words}")


Topic 1: 0.016*"s" + 0.009*"license" + 0.008*"code" + 0.007*"modify" + 0.007*"source" + 0.007*"use" + 0.006*"project" + 0.006*"patent" + 0.006*"c" + 0.005*"bsd"
Topic 2: 0.030*"license" + 0.017*"copyright" + 0.014*"s" + 0.012*"code" + 0.012*"t" + 0.011*"use" + 0.011*"work" + 0.008*"software" + 0.007*"copy" + 0.007*"project"
Topic 3: 0.036*"license" + 0.029*"gpl" + 0.024*"code" + 0.017*"source" + 0.016*"library" + 0.015*"use" + 0.013*"work" + 0.012*"s" + 0.011*"software" + 0.011*"program"
Topic 4: 0.025*"license" + 0.011*"s" + 0.011*"work" + 0.011*"patent" + 0.011*"cc" + 0.010*"t" + 0.009*"use" + 0.007*"source" + 0.007*"software" + 0.007*"sa"
Topic 5: 0.019*"code" + 0.015*"t" + 0.015*"s" + 0.013*"company" + 0.013*"source" + 0.012*"software" + 0.010*"work" + 0.008*"use" + 0.008*"project" + 0.008*"license"


In [ ]:
!pip install pyLDAvis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.7 MB/s eta 0:00:00


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare visualization
lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pandas as pd

# Load dataset
df = pd.read_csv("processed_merged_ianal.csv")

# Tokenize text
tokenized_texts = df["cleaned_text"].apply(lambda x: x.split())

# Create dictionary and corpus for LDA
dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Define number of topics
num_topics = 5

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10)

# Display topics with top words
topics = lda_model.print_topics(num_words=10)
for topic_id, topic_words in topics:
    print(f"Topic {topic_id+1}: {topic_words}")


Topic 1: 0.015*"user" + 0.010*"datum" + 0.008*"s" + 0.007*"t" + 0.006*"app" + 0.005*"company" + 0.005*"information" + 0.005*"ianal" + 0.005*"server" + 0.005*"device"
Topic 2: 0.041*"license" + 0.023*"code" + 0.022*"library" + 0.016*"source" + 0.016*"gpl" + 0.015*"use" + 0.013*"lgpl" + 0.011*"work" + 0.009*"link" + 0.009*"s"
Topic 3: 0.026*"code" + 0.021*"gpl" + 0.020*"source" + 0.016*"software" + 0.015*"license" + 0.014*"program" + 0.014*"s" + 0.012*"t" + 0.010*"modify" + 0.009*"work"
Topic 4: 0.024*"license" + 0.014*"t" + 0.013*"s" + 0.011*"use" + 0.011*"copyright" + 0.010*"software" + 0.009*"source" + 0.008*"work" + 0.008*"code" + 0.007*"file"
Topic 5: 0.027*"license" + 0.018*"code" + 0.017*"s" + 0.016*"work" + 0.012*"gpl" + 0.012*"t" + 0.012*"use" + 0.011*"project" + 0.008*"company" + 0.008*"source"


In [ ]:
from textblob import TextBlob

# Function to get sentiment score (-1 to 1)
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Apply sentiment analysis
df["sentiment_score"] = df["cleaned_text"].apply(get_sentiment)

# Categorize sentiment
df["sentiment_label"] = df["sentiment_score"].apply(lambda x: "Negative" if x < 0 else "Neutral" if x == 0 else "Positive")

# Print sentiment distribution
print(df["sentiment_label"].value_counts())

# Save results
df.to_csv("ianal_sentiment_analysis.csv", index=False)


sentiment_label
Positive    497
Negative     78
Neutral       1
Name: count, dtype: int64


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Ensure VADER is downloaded
nltk.download("vader_lexicon")

# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to get VADER sentiment score
def get_vader_sentiment(text):
    sentiment_score = sia.polarity_scores(text)["compound"]  # Compound score (-1 to 1)

    # Custom thresholds for better descriptive labels
    if sentiment_score >= 0.5:
        return "Confident"
    elif sentiment_score > 0.1:
        return "Positive but Cautious"
    elif sentiment_score < -0.1:
        return "Uncertain/Cautious"
    elif sentiment_score <= -0.5:
        return "Strongly Negative"
    else:
        return "Neutral"

# Apply the function
df["vader_sentiment_label"] = df["cleaned_text"].apply(get_vader_sentiment)

# Print updated sentiment breakdown
print(df["vader_sentiment_label"].value_counts())

# Save new sentiment labels
df.to_csv("ianal_vader_sentiment_analysis.csv", index=False)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


vader_sentiment_label
Confident                499
Positive but Cautious     35
Uncertain/Cautious        34
Neutral                    8
Name: count, dtype: int64
